In [1]:
pip install -U --user shodan

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install azure-cosmos

In [3]:
import requests, os, json
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
from datetime import datetime

In [4]:
def clasificar_severidad_cvss(cvss_score):
    # Verifica si el puntaje es None o la cadena 'null'
    if cvss_score is None or cvss_score == 'null':
        return 'Unknown'
    
    # Asegura que el puntaje sea un float para manejar correctamente la clasificación
    try:
        cvss_score = float(cvss_score)
    except ValueError:
        return 'Unknown'

    if cvss_score == 0:
        return 'None'
    elif 0.1 <= cvss_score <= 3.9:
        return 'Low'
    elif 4.0 <= cvss_score <= 6.9:
        return 'Medium'
    elif 7.0 <= cvss_score <= 8.9:
        return 'High'
    elif 9.0 <= cvss_score <= 10.0:
        return 'Critical'
    else:
        return 'Unknown'


In [5]:
def obtener_info_cve(cve_id):
    url = f'https://cveawg.mitre.org/api/cve/{cve_id}'
    response = requests.get(url)
    if response.status_code == 200:
        response_json = response.json()
        descripcion = response_json["containers"]["cna"]["descriptions"][0]["value"]
        return descripcion
    else:
        print(f"Error al obtener información para {cve_id}: {response.status_code}")
        return None

In [ ]:
# Define un diccionario llamado "settings" que contiene la configuración para Azure Cosmos DB
settings = {
    'host': os.environ.get('ACCOUNT_HOST', ''),  # Obtiene el valor del host de la variable de entorno ACCOUNT_HOST, si no está definida, utiliza un valor predeterminado
    'master_key': os.environ.get('ACCOUNT_KEY', ''),  # Obtiene el valor de la clave maestra de la variable de entorno ACCOUNT_KEY, si no está definida, utiliza un valor predeterminado
    'database_id': os.environ.get('COSMOS_DATABASE', ''),  # Obtiene el nombre de la base de datos de la variable de entorno COSMOS_DATABASE, si no está definida, utiliza un valor predeterminado
    'container_id': os.environ.get('COSMOS_CONTAINER','')  # Obtiene el nombre del contenedor de la variable de entorno COSMOS_CONTAINER, si no está definida, utiliza un valor predeterminado
}

# Asigna los valores de la configuración a variables individuales para facilitar su uso
api_key = "652BXWTonWlRS96a9GwboXONsiSwtkaT"
HOST = settings['host']  # La URL del servicio de Azure Cosmos DB
MASTER_KEY = settings['master_key']  # La clave maestra para acceder al servicio de Azure Cosmos DB
DATABASE_ID = settings['database_id']  # El ID de la base de datos en Azure Cosmos DB
CONTAINER_ID = settings['container_id']  # El ID del contenedor en Azure Cosmos DB

In [ ]:
def read_item(container, doc_id, account_number):
    """
    Esta función lee un ítem de la base de datos de Azure Cosmos DB utilizando el ID del documento y el número de cuenta como clave de partición.

    Parámetros:
    - container: El cliente del contenedor de Azure Cosmos DB.
    - doc_id: El ID del documento que se desea leer.
    - account_number: El número de cuenta que se utilizará como clave de partición.

    No retorna nada, pero imprime la información del ítem leído.
    """
    print('\nReading Item by Id\n')
    # Hace una búsqueda eficiente del ítem utilizando la clave de partición y el ID
    response = container.read_item(item=doc_id, partition_key=account_number)
    print('Item read by Id {0}'.format(doc_id))
    print('Partition Key: {0}'.format(response.get('partitionKey')))
    print('Description: {0}'.format(response.get('VulnerabilityDesc')))

# Creación del cliente para Azure Cosmos DB
client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)

# Intenta crear la base de datos, si ya existe, obtiene la instancia de la misma
try:
    db = client.create_database(id=DATABASE_ID)
    print(f"Database with id '{DATABASE_ID}' created.")
except exceptions.CosmosResourceExistsError:
    db = client.get_database_client(DATABASE_ID)
    print(f"Database with id '{DATABASE_ID}' already exists.")

# Intenta crear el contenedor, si ya existe, obtiene la instancia del mismo
try:
    container = db.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path='/partitionKey'))
    print(f"Container with id '{CONTAINER_ID}' created.")
except exceptions.CosmosResourceExistsError:
    container = db.get_container_client(CONTAINER_ID)
    print(f"Container with id '{CONTAINER_ID}' already exists.")

In [9]:
from datetime import datetime
from random import randint  # Importa la función randint para generar números aleatorios

def secDataFormat(ip_address, datos):
    # Genera un número aleatorio entre 1 y 1000 para usar como partitionKey
    partitionKey_random = randint(1, 1000)
    
    current_date = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
    data_combined = {
        "id": f"{ip_address}",
        "partitionKey": str(partitionKey_random),  # Usa el número aleatorio como partitionKey
        "vulns": datos
    }
    return data_combined


In [11]:
def main(ip_addresses):
    for ip_address in ip_addresses:
        try:
            resultado_dict = secDataFormat(ip_address,obtener_vulnerabilidades_shodan(ip_address))  # Agrega el argumento api_key si es necesario
            if resultado_dict:  # Verifica si se obtuvieron resultados
                print(resultado_dict)
                response = container.upsert_item(body=resultado_dict)
                print("Ítem insertado o actualizado exitosamente.")
            else:
                print(f"No se encontraron vulnerabilidades para la dirección IP {ip_address}")
        except Exception as e:
            print(f"Ocurrió un error al intentar insertar o actualizar el ítem para la dirección IP {ip_address}: {e}")

# Ejemplo de uso:
# Reemplaza 'tu_api_key' con tu clave de API de Shodan
# Reemplaza 'ip_address_a_consultar' con la dirección IP que deseas consultar
#ip_addresses = ["219.75.232.46", "80.120.238.18"]
main(ip_address)



{'id': '219.75.232.46', 'partitionKey': '229', 'vulns': {'CVE-2006-6578': {'CVSS': 'High', 'DESCRIPTION': 'Microsoft Internet Information Services (IIS) 5.1 permits the IUSR_Machine account to execute non-EXE files such as .COM files, which allows attackers to execute arbitrary commands via arguments to any .COM file that executes those arguments, as demonstrated using win.com when it is in a web directory with certain permissions.'}, 'CVE-2009-2521': {'CVSS': 'Medium', 'DESCRIPTION': 'Stack consumption vulnerability in the FTP Service in Microsoft Internet Information Services (IIS) 5.0 through 7.0 allows remote authenticated users to cause a denial of service (daemon crash) via a list (ls) -R command containing a wildcard that references a subdirectory, followed by a .. (dot dot), aka "IIS FTP Service DoS Vulnerability."'}, 'CVE-2009-4445': {'CVSS': 'Medium', 'DESCRIPTION': 'Microsoft Internet Information Services (IIS), when used in conjunction with unspecified third-party upload ap